In [1]:
# import dependencies
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import requests
import pymongo
import time

C:\Users\dudee_000\anaconda\Anaconda3\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


In [4]:
# Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text. 
url='https://mars.nasa.gov/news/'

# Requests pulls up different html than browser after scripts run
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

browser.visit(url)

time.sleep(5)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

result = soup.find('div',class_='list_text')

browser.quit()


# Assign the text to variables that you can reference later.
news_title = result.find('a').text
news_p = result.find('div',class_='article_teaser_body').text

In [7]:
print(news_title)
print(news_p)

Curiosity on the Move Again
NASA's Mars Curiosity rover drove about 197 feet over the weekend to a site called Lake Orcadie, pushing its total odometry to over 12 miles.


In [2]:
# Visit the url for JPL Featured Space Image here.
url='https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
# Use splinter to navigate the site and find the image url for the current Featured Mars Image 
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

browser.visit(url)

browser.click_link_by_partial_text('FULL IMAGE')
time.sleep(2) #allow time for button to appear
browser.click_link_by_partial_text('more info')

html=browser.html
soup = BeautifulSoup(html, 'html.parser')

result=soup.find('figure', class_='lede')
imgsource=result.find('a')['href']

browser.quit()


# assign the url string to a variable called featured_image_url.
featured_image_url='https://www.jpl.nasa.gov'+imgsource
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18848_hires.jpg'

In [3]:
#  Visit the Mars Weather twitter account
# scrape the latest Mars weather tweet from the page.
# Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text.
url='https://twitter.com/marswxreport?lang=en'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'html'
soup = BeautifulSoup(response.text, 'html')

result = soup.find('p',class_='tweet-text')

# Save the tweet text for the weather report as a variable called mars_weather.
mars_weather = result.text
mars_weather

'Sol 2224 (2018-11-08), high 0C/32F, low -72C/-97F, pressure at 8.65 hPa, daylight 06:19-18:36'

In [12]:
# Visit the Mars Facts webpage
# use Pandas to scrape the table containing facts about the planet
# including Diameter, Mass, etc.
url = 'https://space-facts.com/mars/'

table = pd.read_html(url)
mars_df = table[0]
mars_df.columns = ['','Data']
mars_df.set_index('',inplace=True)
mars_df

,Data
,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [17]:
# Use Pandas to convert the data to a HTML table string.
mars_table = mars_df.to_html()
mars_table.replace('\n','')
with open('mars_table.html', 'w') as fo:
    fo.write(mars_table)

In [18]:
# Visit the USGS Astrogeology site to obtain
# high resolution images for each of Mar's hemispheres.
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
img_base = 'https://astrogeology.usgs.gov'

hemispheres = ["Valles Marineris Hemisphere",
               "Cerberus Hemisphere",
               "Schiaparelli Hemisphere",
               "Syrtis Major Hemisphere"]
hemisphere_image_urls = []

executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

for hemisphere in hemispheres:
#     You will need to click each of the links to the hemispheres
#     in order to find the image url to the full resolution image.
    browser.visit(url)
    browser.click_link_by_partial_text(hemisphere)
    
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    result = soup.find('img',class_='wide-image')['src']
    img_url = img_base + result
#     Append the dictionary with the image url string and the hemisphere title to a list.
#     This list will contain one dictionary for each hemisphere.
    entry={'title': hemisphere, 'img_url': img_url}
    hemisphere_image_urls.append(entry)
    
browser.quit()

[{'title': 'Valles Marineris Hemisphere',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'},
 {'title': 'Cerberus Hemisphere',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'}]